In [18]:
# 載入所需套件

import requests
from bs4 import BeautifulSoup
import json
import datetime
import time
import pandas as pd

In [19]:
###############################################################################
#                     股票機器人 標準差＆本益比分析選股（簡易板）                    #
###############################################################################

# 抓到今天的時間
now = datetime.datetime.now()

# 先與網站請求抓到價格
def getstock(stocknumber):
    url = 'http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20180814&stockNo=' + stocknumber + '&_=' + str(time.mktime(now.timetuple()))
    list_req = requests.get(url)
    soup = BeautifulSoup(list_req.content, "html.parser")
    getjson=json.loads(soup.text)
    
    # 判斷請求是否成功
    if getjson['stat'] != '很抱歉，沒有符合條件的資料!': 
        return [getjson['data']]
    else:
        return [] # 請求失敗回傳空值

In [20]:
# 開始計算股票的平均以及標準差＆抓取指定股票之本益比，並從參考數值判斷是否購買

def Standard_Deviation(stocknumber):    
    # 第一步 載入所需套件 
    import requests
    import pandas as pd
    import numpy as np
    from io import StringIO
    
    # 第二步 爬取股票資訊（含本益比）
    stocklist = getstock(stocknumber)
    date = str(now.date())
    r = requests.post('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + date + '&type=ALL')

    # 第三步 篩選出個股盤後資訊
    str_list = []
    for i in r.text.split('\n'):
        if len(i.split('",')) == 17 and i[0] != '=':       
            i = i.strip(",\r\n")
            str_list.append(i)      

    # 第四步 印出選股資訊
    df = pd.read_csv(StringIO("\n".join(str_list)))  
    pd.set_option('display.max_rows', None)
    
    n = len(df['證券代號'])
    for i in range(n):
        if df['證券代號'][i]==str(stocknumber):
            print("該支股票今日本益比為：",df['本益比'][i],"\n")
    
    # 判斷是否為空值
    if len(stocklist) != 0:
        stockdf = pd.DataFrame(stocklist[0],columns=["日期","成交股數","成交金額","開盤價","最高價","最低價","收盤價","漲跌價差","成交筆數"])
        stockAverage = pd.to_numeric(stockdf['收盤價']).mean()
        stockSTD = pd.to_numeric(stockdf['收盤價']).std()
        
        # 看看這隻股票現在是否便宜（平均-兩倍標準差）
        buy='\n很貴不要買'
        if pd.to_numeric(stockdf['收盤價'][-1:]).values[0] < stockAverage - (2*stockSTD):
            buy = '\n這隻股票現在很便宜喔！'
            
        # 顯示結果
        print('收盤價 ＝ ' + stockdf['收盤價'][-1:].values[0])
        print('中間價 ＝ ' + str(stockAverage))
        print('線距 ＝ ' + str(stockSTD))
        print(buy)
    else:
        print('請求失敗，請檢查您的股票代號')

In [21]:
Standard_Deviation(stocknumber='2330')

該支股票本益比為： 24.57 

收盤價 ＝ 256.00
中間價 ＝ 245.80434782608697
線距 ＝ 6.249980237122904

很貴不要買


In [115]:
stocknumber='2330'
url = 'http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20180814&stockNo=' + stocknumber + '&_=' + str(time.mktime(now.timetuple()))
list_req = requests.get(url)
soup = BeautifulSoup(list_req.content, "html.parser")
getjson=json.loads(soup.text)
if len(stocklist) != 0:
    stockdf = pd.DataFrame(stocklist[0],columns=["日期","成交股數","成交金額","開盤價","最高價","最低價","收盤價","漲跌價差","成交筆數"],index=stockdf['日期'])

In [119]:
stockdf.tail()

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
日期,,,,,,,,,
107/08/27,107/08/27,"15,579,299","3,825,736,755",246.00,246.50,244.50,245.00,+1.50,"7,858"
107/08/28,107/08/28,"40,410,531","10,075,613,779",248.50,250.00,248.00,249.50,+4.50,"16,881"
107/08/29,107/08/29,"57,633,182","14,784,556,456",253.50,259.00,253.50,259.00,+9.50,"25,668"
107/08/30,107/08/30,"52,622,808","13,914,404,873",264.50,268.00,262.50,263.50,+4.50,"22,862"
107/08/31,107/08/31,"70,000,506","17,920,548,387",257.50,259.00,254.00,256.00,-7.50,"19,019"
